In [ ]:
import numpy as np
from astropy.table import Table, join

from bokeh.io import output_notebook, output_file, show, save
from bokeh.plotting import figure
from bokeh.palettes import Category20


## Combine summary tables from each fit run

In [ ]:
all_ia = Table.read('sncosmo_results/snia_ugriz.csv')
blue_ia = Table.read('sncosmo_results/snia_ug.csv')
red_ia = Table.read('sncosmo_results/snia_riz.csv')
all_91bg = Table.read('sncosmo_results/91bg_ugriz.csv')
blue_91bg = Table.read('sncosmo_results/91bg_ug.csv')
red_91bg = Table.read('sncosmo_results/91bg_riz.csv')

tables = (blue_ia, red_ia, blue_91bg, red_91bg)
table_names = ('blue_ia', 'red_ia', 'blue_91bg', 'red_91bg')
combined_table = None
for table, name in zip(tables, table_names):
    # Rename columns
    temp_table = table.copy()
    for column in temp_table.colnames:
        if column in ('cid', 'class'):
            continue
            
        temp_table.rename_column(column, column + '_' + name)
        
    # Join tables
    if combined_table is None:
        combined_table = temp_table
        continue
        
    combined_table = join(combined_table, temp_table, 
                          join_type='outer', 
                          keys=['cid', 'class'])
    
del temp_table


In [ ]:
print('Total number targets (ugriz):', len(all_ia))
print('Number of fit ia (blue): {} / {}'.format(
    np.sum(~np.isnan(blue_ia['chi'])), len(blue_ia)))

print('Number of fit ia (red): {} / {}'.format(
    np.sum(~np.isnan(red_ia['chi'])), len(red_ia)))

print('Number of fit 91 (blue): {} / {}'.format(
    np.sum(~np.isnan(blue_91bg['chi'])), len(blue_91bg)))

print('Number of fit 91 (red): {} / {}'.format(
    np.sum(~np.isnan(red_91bg['chi'])), len(red_91bg)))


## Chi-squared Comparison

In [ ]:
# Get plot data
class_set = sorted(set(combined_table['class']))
x_vals = (combined_table['chi_blue_ia'] / combined_table['dof_blue_ia']
          - combined_table['chi_blue_91bg'] / combined_table['dof_blue_91bg'])

y_vals = (combined_table['chi_red_ia'] / combined_table['dof_red_ia'] 
          - combined_table['chi_red_91bg'] / combined_table['dof_red_91bg'])

# Plot data
fig = figure(plot_width=600, plot_height=400, 
             tools=['save', 'pan', 'box_zoom', 'reset', 'wheel_zoom'])

for classification, color in zip(class_set, Category20[20]):
    indices = combined_table['class'] == classification
    fig.circle(x_vals[indices], y_vals[indices], color=color, legend=classification)

# Move legend outside the plot
fig.legend.click_policy="hide"
new_legend = fig.legend[0]
fig.legend[0].plot = None
fig.add_layout(new_legend, 'left')
fig.yaxis.axis_label = "Chi2 Ia - Chi2 91bg (riz)"
fig.xaxis.axis_label = "Chi2 Ia - Chi2 91bg (ug)"

output_notebook()
show(fig)

# output_file("chi_squared_comparison.html")
# save(fig)
